# Segmenting and Clustering Neighborhoods in Toronto by the following criteria:
### Question 1: Web scrape data from Wikipedia and create custom data frame (borough & neighborhood)
### Question 2: Get the latitude and the longitude coordinates of each neighborhood and add them to dataframe
### Question 3: Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto

#### Installing BeautifulSoup to scrape data neighborhood data from Toronto (Wikipedia) along with pandas and numpy (Note: Data is from 2 January, 2020)

In [84]:
import requests
URL = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=935851093').text

In [85]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(URL,'lxml')
# print(soup.prettify())

#### Tabular data can be found by looking for classname = 'wikitable sortable', and hence that is the only data to extract

In [86]:
table = soup.find('table',{'class':'wikitable sortable'})
# table

In [87]:
print(table.tr.text)


Postcode
Borough
Neighborhood



In [88]:
headers="Postcode,Borough,Neighbourhood"

#### Getting all values from tuples (tr) and separating data (td) with delimiter ','

In [89]:
table1 = ""
for tr in table.find_all('tr'):
    row1 = ""
    for tds in tr.find_all('td'):
        row1 = row1 + "," + tds.text
    table1 = table1 + row1[1:]
# print(table1)

#### Writing data into .csv format for convenience, and then convert to dataframe and perform formatting operations

In [90]:
file = open("toronto.csv","wb")
file.write(bytes(table1,encoding="ascii",errors="ignore"))

8708

In [91]:
import pandas as pd
df = pd.read_csv('toronto.csv',header=None)
df.columns = ["Postalcode","Borough","Neighbourhood"]

In [92]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Find tuples with Borough = 'Not Assigned', and then drop them

In [93]:
# Get names of indexes for which column Borough has value "Not assigned"
indexNames = df[ df['Borough'] =='Not assigned'].index

# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

In [94]:
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [95]:
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']
df.head(10)

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [103]:
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)
df_new=result.reset_index()

### Answer to Question 1 with .shape requirement

In [104]:
df_new.head(15)


(103, 3)